In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import time
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True,precision=6)
#tf.reset_default_graph()

OP_NONE=-1
OP_BUY=0
OP_SELL=1
ORDER_STATE_OPENED=0
ORDER_STATE_CLOSED=1
POINT=100000
TRADERS_NUM=20
CROSS_PROBALBILITY=0.8
MUTATION_PROBALBILITY=0.05
INITIAL_FOUNDS=1000

def get_datafame(fname):
    df=pd.read_csv(fname,header=None)
    df.columns=['date','time','open','high','low','close','volume']
    df['timestamp']=df['date']+' '+df['time']
    df['timestamp']=df['timestamp'].apply(lambda x:time.mktime(time.strptime(x,'%Y.%m.%d %H:%M'))/time.mktime(time.strptime('2082-02-11 00:00','%Y-%m-%d %H:%M')))
    df['co']=(df['close']-df['open'])*POINT
    return df

def get_data(fname):
    df=get_datafame(fname)
    xi=[];yi=[]
    for i in range(PERIOD,PERIOD+2):
    #for i in range(PERIOD,len(df)):
        xi.append(df['co'][i-PERIOD:i].tolist())
        yi.append(df.iloc[i]['co'])
    xi=np.array(xi)
    return (xi,yi)

class Market(object):
    def __init__(self,spread,series,ntraders):
        self.series=series
        self.ntraders=ntraders
        self.traders=[Trader(self,INITIAL_FOUNDS) for i in range(0,ntraders)]
        self.bar=0
        self.sess=tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def move(self):
        for bar in range(0,len(self.series)):
            #print('bar:%f'%(bar))
            self.bar=bar
            self.price=self.series.iloc[bar]['close']
            self.checkOrders()
            #print('before ntraders:%d mean:%f'%(len(self.traders),np.array([trader.accountEquity() for trader in self.traders]).mean()))
            #print(np.array([t.accountEquity() for t in self.traders]))
            #mean=np.array([t.accountEquity() for t in self.traders]).mean()
            #print(mean)
            self.traders=[t for t in self.traders if t.accountEquity()>0]
            print('after ntraders:%d mean:%f'%(len(self.traders),np.array([trader.accountEquity() for trader in self.traders]).mean()))
            if len(self.traders)==0:
                self.crossTraders()
            #self.traders=filter(lambda x:x.accountEquity()>0,self.traders)
            for trader in self.traders:
                trader.makeDecision()
                
    def checkOrders(self):
        #high=int(self.series.iloc[self.bar]['high']*POINT)
        #low=int(self.series.iloc[self.bar]['low']*POINT)
        #if high>low:ticks=np.random.randint(low,high,10)/POINT
        #else: ticks=[low/POINT]
        high=self.series.iloc[self.bar]['high']
        low=self.series.iloc[self.bar]['low']
        ticks=[low,high]
        for price in ticks:
            self.price=price
            for trader in self.traders:
                for order in trader.orders:
                    #print('order:%d'%(order.ticket))
                    if order.op==OP_BUY:
                        if order.stoploss!=0 and price<=order.stoploss:
                            order.close('buy ticket:%d trader:%d stoploss op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                        if order.takeprofit!=0 and price>=order.takeprofit:
                            order.close('buy ticket:%d trader:%d takeprofit op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                    if order.op==OP_SELL:
                        if order.stoploss!=0 and price>=order.stoploss:
                            order.close('sell ticket:%d trader:%dstoploss op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                        if order.takeprofit!=0 and price<=order.takeprofit:
                            order.close('sell ticket:%d trader:%d takeprofit op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                trader.orders=[order for order in trader.orders if order.state==ORDER_STATE_OPENED]
                #trader.orders=filter(lambda x:x.state==ORDER_STATE_OPENED,trader.orders)
    
    def crossTraders(self):
        print("============cross traders==================")
        if len(self.traders)==0:
            self.traders=[Trader(self,INITIAL_FOUNDS) for i in range(0,self.ntraders)]
            return
        while(len(self.traders)<self.ntraders):
            male=self.traders[np.random.randint(0,len(self.traders))]
            female=self.traders[np.random.randint(0,len(self.traders))]
            child=male.cross(female)
            self.traders.append(child)
        for trader in self.traders:
            trader.orders=[]
            trader.balance=INITIAL_FOUNDS
    def report(self):
        pass
        #index=0
        #for trader in sorted(self.traders, key=lambda x:x.profit, reverse=True):
        #    print(trader.profit)
        #    index+=1
        #    if index>3:break
traderNo=0
class Trader(object):
    def __init__(self,market,fouds,period=None,weights_in=None):
        global traderNo;
        self.traderNo=traderNo
        traderNo+=1
        hidden_num=30
        self.market=market
        self.balance=fouds
        self.equity=fouds
        self.margin=0
        self.profit=0
        self.orders=[]
        if period==None:self.period=np.random.randint(5,200)
        else:self.period=period
        #self.weights_in=tf.Variable(tf.truncated_normal(shape=[hidden_num,self.period],stddev=1.0/tf.sqrt(self.period)))
        #self.weights_hidden
        self.weights_in=np.random.randn(hidden_num,self.period)
        self.weights_hidden=np.random.randn(hidden_num)
        self.bias_hidden=np.random.randn(hidden_num,1)
        self.weights_out=np.random.randn(3,hidden_num)
        #stddev=1.0/tf.sqrt(tf.cast(self.period,tf.float32))
        #if weights_in==None:self.weights_in=tf.Variable(tf.truncated_normal(shape=[hidden_num,self.period],stddev=stddev))
        #else:self.weights_in=weights_in
        #self.weights_hidden=tf.Variable(tf.truncated_normal(shape=[hidden_num],stddev=stddev))
        #self.bias_hidden=tf.Variable(tf.truncated_normal(shape=[hidden_num,1]))
        #self.weights_out=tf.Variable(tf.truncated_normal(shape=[3,hidden_num]))
        self.x=tf.placeholder(shape=[self.period,1],dtype=tf.float64)
        
        y=tf.matmul(self.weights_in,self.x)
        y=tf.add(y,self.bias_hidden)
        #print('add')
        #print(self.market.sess.run(x))
        y=tf.nn.sigmoid(y)
        y=tf.matmul(self.weights_out,y)
        #print('matmul')
        #print(self.market.sess.run(x))
        y=tf.reshape(y,(3,))
        y=tf.nn.softmax(y)
        y=tf.argmax(y)
        self.y=y
    
    def pretrain(self):
        pass;
        
    def sigmoid(self,x):  
        return 1.0 / (1 + np.exp(-x))
    
    def softmax(self,x):
        return np.exp(x)/np.sum(np.exp(x),axis=0)
        
    
    def makeDecision(self):
        if self.market.bar<self.period:
            return
        xi=np.array(self.market.series[self.market.bar-self.period:self.market.bar]['close'])[:,np.newaxis]
        op=self.market.sess.run(self.y,feed_dict={self.x:xi})-1
        #print('op: %d'%(op))
        if op==OP_NONE:return
        if len(self.orders)>0:return
        lots=1
        takeprofit=0.001
        stoploss=0.001
        #lots=round(np.clip(o2[1],0.1,0.5),2)
        #takeprofit=round(np.clip(o2[2],0,500.0/POINT),5)
        #stoploss=round(np.clip(o2[3],0,500.0/POINT),5)
        #print('op:%d lots:%f takeprofit:%f stoploss:%f'%(op,lots,takeprofit,stoploss))
        if lots==0: return
        if op==OP_BUY:
            takeprofit=self.market.price+takeprofit
            stoploss=self.market.price-stoploss
            self.buy(lots,takeprofit,stoploss)
        if op==OP_SELL:
            takeprofit=self.market.price-takeprofit
            stoploss=self.market.price+stoploss
            self.sell(lots,takeprofit,stoploss)
        
    def buy(self,lots,takeprofit,stoploss):
        order=Order(self.market,self)
        order.open(OP_BUY,lots,takeprofit,stoploss)
        #self.orders.append(order)
    def sell(self,lots,takeprofit,stoploss):
        order=Order(self.market,self)
        order.open(OP_SELL,lots,takeprofit,stoploss)
        #self.orders.append(order)
    def accountBalance(self):
        return self.balance
    def accountProfit(self):
        return sum([order.orderProfit() for order in self.orders if order.state==ORDER_STATE_OPENED])
        #return sum(filter(lambda x:x.state==ORDER_STATE_OPENED,self.orders))
    def accountEquity(self):
        #print("no:%d profit:%f"%(self.traderNo,self.accountProfit()))
        return self.balance+self.accountProfit()
    def orders():
        return self.orders
    def cross(self,female):
        child=Trader(self.market,INITIAL_FOUNDS,self.period,self.weights_in)
        #child.period=self.period
        #child.weights_in=self.weights_in
        #self.market.sess.run(tf.assign(child.weights_in,self.weights_in))
        self.market.sess.run(
            tf.assign(child.weights_hidden,(CROSS_PROBALBILITY*self.weights_hidden+(1-CROSS_PROBALBILITY)*female.weights_hidden)))
        child.mutate()
        return child
    def mutate(self):
        self.market.sess.run(tf.assign(self.weights_hidden,tf.nn.dropout(self.weights_hidden,1-MUTATION_PROBALBILITY)))
    #def dropout(self,weights,prob):
    #    mask=np.random.random(weights.shape)
    #    mask=mask<(1-prob)
    #    return weights*mask
    
ticket=0
class Order(object):
    def __init__(self,market,trader):
        global ticket
        self.market=market
        self.trader=trader
        self.ticket=ticket
        ticket+=1
        
    def open(self,op,lots,takeprofit,stoploss):
        self.op=op
        self.openPrice=self.market.price
        self.lots=lots
        self.takeprofit=takeprofit
        self.stoploss=stoploss
        self.trader.orders.append(self)
        self.state=ORDER_STATE_OPENED
        #market.orders.append(self)
    def orderProfit(self):
        if self.op==OP_SELL:
            return self.lots*(self.openPrice-self.market.price)*POINT
        if self.op==OP_BUY:
            return self.lots*(self.market.price-self.openPrice)*POINT
    def close(self,notes):
        profit=self.orderProfit()
        self.trader.balance+=profit
        self.state=ORDER_STATE_CLOSED
        #print(notes)
        #print('op:%d openPrice:%f price:%f profit:%f takeprofit:%f stoploss:%f'%(self.op,self.openPrice,self.trader.market.price,profit
        #                                                                         ,self.takeprofit,self.stoploss))
        #self.traders.orders.remove(self)
        #self.market.orders.remove(self)
fname=''
def main():
    market=Market(2,get_datafame(fname),TRADERS_NUM)
    for i in range(0,EPOCH):
        market.move()
        market.crossTraders()
        market.report()


In [22]:
TRADERS_NUM=10
EPOCH=1
fname='test.csv'
#main()
import cProfile
cProfile.run('main()')

after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 mean:1000.000000
after ntraders:10 me

after ntraders:10 mean:985.400000
after ntraders:10 mean:983.700000
after ntraders:10 mean:982.900000
after ntraders:10 mean:983.400000
after ntraders:10 mean:983.600000
after ntraders:10 mean:984.400000
after ntraders:10 mean:985.000000
after ntraders:10 mean:985.000000
after ntraders:10 mean:984.600000
after ntraders:10 mean:984.200000
after ntraders:10 mean:984.400000
after ntraders:10 mean:983.500000
after ntraders:10 mean:982.800000
after ntraders:10 mean:982.800000
after ntraders:10 mean:982.800000
after ntraders:10 mean:982.900000
after ntraders:10 mean:983.700000
after ntraders:10 mean:983.700000
after ntraders:10 mean:983.400000
after ntraders:10 mean:982.100000
after ntraders:10 mean:982.100000
after ntraders:10 mean:981.500000
after ntraders:10 mean:981.400000
after ntraders:10 mean:981.200000
after ntraders:10 mean:981.600000
after ntraders:10 mean:980.200000
after ntraders:10 mean:980.700000
after ntraders:10 mean:980.400000
after ntraders:10 mean:981.200000
after ntraders

after ntraders:10 mean:1001.400000
after ntraders:10 mean:1001.200000
after ntraders:10 mean:999.800000
after ntraders:10 mean:999.100000
after ntraders:10 mean:998.700000
after ntraders:10 mean:999.400000
after ntraders:10 mean:1000.800000
after ntraders:10 mean:1001.200000
after ntraders:10 mean:1000.800000
after ntraders:10 mean:999.700000
after ntraders:10 mean:997.200000
after ntraders:10 mean:998.700000
after ntraders:10 mean:998.000000
after ntraders:10 mean:996.500000
after ntraders:10 mean:996.700000
after ntraders:10 mean:996.600000
after ntraders:10 mean:996.700000
after ntraders:10 mean:998.200000
after ntraders:10 mean:999.500000
after ntraders:10 mean:1001.000000
after ntraders:10 mean:1002.900000
after ntraders:10 mean:1003.100000
after ntraders:10 mean:1004.000000
after ntraders:10 mean:1003.600000
after ntraders:10 mean:1002.400000
after ntraders:10 mean:1002.900000
after ntraders:10 mean:1002.900000
after ntraders:10 mean:1003.100000
after ntraders:10 mean:1001.900000

after ntraders:10 mean:991.000000
after ntraders:10 mean:990.700000
after ntraders:10 mean:990.200000
after ntraders:10 mean:989.400000
after ntraders:10 mean:989.000000
after ntraders:10 mean:989.300000
after ntraders:10 mean:990.400000
after ntraders:10 mean:991.300000
after ntraders:10 mean:992.500000
after ntraders:10 mean:992.400000
after ntraders:10 mean:991.500000
after ntraders:10 mean:991.900000
after ntraders:10 mean:990.400000
after ntraders:10 mean:989.800000
after ntraders:10 mean:989.500000
after ntraders:10 mean:990.400000
after ntraders:10 mean:990.700000
after ntraders:10 mean:990.700000
after ntraders:10 mean:990.600000
after ntraders:10 mean:990.000000
after ntraders:10 mean:990.100000
after ntraders:10 mean:992.000000
after ntraders:10 mean:992.000000
after ntraders:10 mean:991.900000
after ntraders:10 mean:992.500000
after ntraders:10 mean:993.700000
after ntraders:10 mean:994.600000
after ntraders:10 mean:994.600000
after ntraders:10 mean:994.400000
after ntraders

        7    0.000    0.000    0.000    0.000 common.py:824(is_signed_integer_dtype)
        9    0.000    0.000    0.000    0.000 common.py:85(is_object_dtype)
        6    0.000    0.000    0.000    0.000 common.py:873(is_unsigned_integer_dtype)
        1    0.000    0.000    0.000    0.000 common.py:94(_is_s3_url)
      261    0.002    0.000    0.051    0.000 common_shapes.py:588(call_cpp_shape_fn)
      191    0.004    0.000    0.044    0.000 common_shapes.py:650(_call_cpp_shape_fn_impl)
      370    0.002    0.000    0.007    0.000 common_shapes.py:656(tensor_to_inference_result)
      191    0.000    0.000    0.008    0.000 common_shapes.py:664(<listcomp>)
      191    0.000    0.000    0.000    0.000 common_shapes.py:666(<listcomp>)
      191    0.000    0.000    0.000    0.000 common_shapes.py:674(<listcomp>)
      191    0.001    0.000    0.002    0.000 common_shapes.py:703(<listcomp>)
      191    0.000    0.000    0.000    0.000 common_shapes.py:706(<listcomp>)
      191    

        1    0.000    0.000    0.000    0.000 parsers.py:397(_validate_names)
        1    0.000    0.000    0.005    0.005 parsers.py:423(_read)
        1    0.000    0.000    0.005    0.005 parsers.py:557(parser_f)
        1    0.000    0.000    0.000    0.000 parsers.py:751(__init__)
        1    0.000    0.000    0.000    0.000 parsers.py:820(close)
        1    0.000    0.000    0.000    0.000 parsers.py:823(_get_options_with_defaults)
        1    0.000    0.000    0.000    0.000 parsers.py:862(_check_file_or_buffer)
        1    0.000    0.000    0.000    0.000 parsers.py:879(_clean_options)
        1    0.000    0.000    0.000    0.000 posixpath.py:230(expanduser)
        1    0.000    0.000    0.000    0.000 pywrap_tensorflow_internal.py:1500(TF_NewSessionOptions)
     8559    0.036    0.000    0.065    0.000 range.py:119(_simple_new)
        1    0.000    0.000    0.000    0.000 range.py:146(_validate_dtype)
        5    0.000    0.000    0.000    0.000 range.py:303(equals)
 